# Plate Stiffener with TDNNS formulation
A squared plate with length $L$ is loaded by a uniform pressure force and simple supported at all boundaries. Three different configurations involving diagonal stiffeners: (0) no stiffener, (1) concentric stiffener with height $h=0.508$, and (2) eccentric stiffener with height $h_s= 1.27$. 

The material and geometrical properties are
$$
E = 117.25,\qquad \nu = 0.3,\qquad \kappa=5/6,\qquad L=25.4,\qquad t=0.254,
$$
the maximal surface force is $P_{\max}=1500\times10^{-6}$, and we use the TDNNS method for nonlinear Naghdi shells.

In [ ]:
# Load NGSolve module, and the Open Cascade (OCC) module
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

# Geometry and material parameters
thickness = 0.254
hs1 = 0.508
hs2 = 1.27
L = 25.4
E, nu = 117.25, 0.3
G = E / (2 * (1 + nu))
kappa = 5 / 6

# Maximal surface force
force = 1e-6 * CF((0, 0, 1500))

# Kind of stiffener: 0 = none, 1 = concentric, 2 = eccentric
stiffener = 0

# Create geometry and mesh
f1 = WorkPlane(Axes((0, 0, 0), n=Z, h=X)).Rectangle(L, L).Face()
f1.edges.name = "outer"
f1.faces.name = "domain"

if stiffener == 1:
    f2 = (
        WorkPlane(Axes((0, L, -hs1), n=-(Y + X), h=-Y))
        .Rectangle(L * sqrt(2), 2 * hs1)
        .Face()
    )
    f3 = (
        WorkPlane(Axes((0, 0, -hs1), n=-(Y - X), h=Y))
        .Rectangle(L / sqrt(2), 2 * hs1)
        .Face()
    )
    f4 = (
        WorkPlane(Axes((L / 2, L / 2, -hs1), n=-(Y - X), h=Y))
        .Rectangle(L / sqrt(2), 2 * hs1)
        .Face()
    )
    f2.edges.Max(Y - X).name = "outer"
    f2.edges.Min(Y - X).name = "outer"
    f3.edges.Min(Y + X).name = "outer"
    f4.edges.Max(Y + X).name = "outer"
    shape = Glue([f1, f2, f3, f4])
elif stiffener == 2:
    f2 = WorkPlane(Axes((0, L, 0), n=-(Y + X), h=-Y)).Rectangle(L * sqrt(2), hs2).Face()
    f3 = WorkPlane(Axes((0, 0, 0), n=-(Y - X), h=Y)).Rectangle(L / sqrt(2), hs2).Face()
    f4 = (
        WorkPlane(Axes((L / 2, L / 2, 0), n=-(Y - X), h=Y))
        .Rectangle(L / sqrt(2), hs2)
        .Face()
    )
    f2.edges.Max(Y - X).name = "outer"
    f2.edges.Min(Y - X).name = "outer"
    f3.edges.Min(Y + X).name = "outer"
    f4.edges.Max(Y + X).name = "outer"
    shape = Glue([f1, f2, f3, f4])
else:
    shape = f1

mesh = Mesh(OCCGeometry(shape).GenerateMesh(maxh=2))
Draw(mesh);

At the simple supported boundary we need to fix the displacement by homogeneous Dirichlet boundary conditions.

In [ ]:
# Set polynomial order for displacement field u, the bending moment
# tensor m and hybrid variable hyb have one degree less
order = 3

# Create finite element spaces
# simple supported at outer edges (bbnd ... co-dimension 2 boundary of 3D mesh)
fesU = VectorH1(
    mesh,
    order=order,
    dirichlet_bbnd="outer",
)
fesM = HDivDivSurface(mesh, order=order - 1, discontinuous=True)
fesH = NormalFacetSurface(mesh, order=order - 1, dirichlet_bbnd="")
fesB = HCurl(mesh, order=order - 1, dirichlet_bbnd="")
fes = fesU * fesM * fesB * fesH

# Symbolic trial functions for displacement field u,
# bending moment tensor m, shear gamma and hybrid variable hyb
u, m, gamma, hyb = fes.TrialFunction()
# Trace needed for m, gamma, and hyb as we are on the surface
m, gamma, hyb = m.Trace(), gamma.Trace(), hyb.Trace()

# Regge FESpace to avoid membrane locking
Regge = HCurlCurl(mesh, order=order - 1, discontinuous=True)

# GridFunction to store the solution
gf_solution = GridFunction(fes, name="solution")

We define the unit normal vector $\hat\nu$, edge-tangential vector $\hat \tau$ and the co-normal vector $\hat\mu = \hat\nu\times \hat \tau$ at the initial configuration.

Then the projection operator onto the tangent space, deformation gradient, Cauchy-Green, and Green tensors $\boldsymbol{P}$, $\boldsymbol{F}$, $\boldsymbol{C}$, and $\boldsymbol{E}$ are introduced.

Finally, the unit normal, edge-tangential, and co-normal vectors $\nu, \tau,\mu$ on the deformed configuration are declared, which depend on the unknown displacement field $u$.

In [ ]:
# Surface unit normal, edge-tangential, and co-normal vectors on initial configuration
nv = specialcf.normal(mesh.dim)
tv = specialcf.tangential(mesh.dim)
cnv = Cross(nv, tv)

# Projection to the surface tangent space
Ptau = Id(mesh.dim) - OuterProduct(nv, nv)
# Surface deformation gradient
Ftau = Grad(u).Trace() + Ptau
# Surface Cauchy-Green tensor
Ctau = Ftau.trans * Ftau
# Surface Green-Lagrange strain tensor
Etautau = 0.5 * (Ctau - Ptau)
# surface determinant
J = Norm(Cof(Ftau) * nv)


def PseudoInverse(mat, v):
    """Pseudo Inverse of a rank (n-1) matrix
    v needs to lie in the kernel of mat
    """
    return Inv(mat.trans * mat + OuterProduct(v, v)) * mat.trans


# Surface unit normal, edge-tangential, co-normal vectors, and director on deformed configuration
nv_def = Normalize(Cof(Ftau) * nv)
tv_def = Normalize(Ftau * tv)
cnv_def = Cross(nv_def, tv_def)
director = nv_def + PseudoInverse(Ftau, nv).trans * gamma

# Surface Hessian weighted with director on deformed configuration
H_nv_def = (u.Operator("hesseboundary").trans * director).Reshape((3, 3))

For the angle computation of the bending energy we use an averaged unit normal vector avoiding the necessity of using information of two neighbored element at once (+ a more stable formulation using the co-normal vector instead of the unit normal vector)

<center><img src="pictures/nonsmooth_av_nv_el_nv.png" width="150"> </center>

\begin{align*}
\sum_{E\in\mathcal{E}_h}\int_E(\sphericalangle(\hat{\nu}_L,\hat{\nu}_R)-\sphericalangle(\nu_L,\nu_R)\circ\phi)\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,ds &= \sum_{T\in\mathcal{T}_h}\int_{\partial T}(\sphericalangle(\mu\circ\phi,P^{\perp}_{\tau\circ\phi}(\{\!\{\nu^n\}\!\}))-\sphericalangle(\hat{\mu},\{\!\{\hat{\nu}\}\!\}))\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,ds,
\end{align*}
where 
$$
P^{\perp}_{\tau\circ\phi}(v)= \frac{1}{\|\boldsymbol{P}^{\perp}_{\tau\circ\phi}v\|}\boldsymbol{P}^{\perp}_{\tau\circ\phi}v,\qquad \boldsymbol{P}^{\perp}_{\tau\circ\phi}= \boldsymbol{I}-\tau\circ\phi\otimes\tau\circ\phi
$$
denotes the (nonlinear) normalized projection to the plane perpendicular to the deformed edge-tangential vector $\tau$ for measuring the correct angle.

In [ ]:
# Unit normal vector on current configuration
# Used to update averaged unit normal vector during load-steps
cf_nv_cur = Normalize(Cof(Ptau + Grad(gf_solution.components[0])) * nv)

# FE space for averaged normal vectors living only on the edges of the mesh
fesVF = VectorFacetSurface(mesh, order=order - 1)
# GridFunctions to save averaged normal vectors on deformed and initial configurations
averaged_nv = GridFunction(fesVF)
averaged_nv_init = GridFunction(fesVF)

# Initialize by averaging unit normal vectors on initial configuration
# definedon=mesh.Boundaries(".*") is needed as we interpolate on the surface mesh
averaged_nv.Set(nv, dual=True, definedon=mesh.Boundaries(".*"))
averaged_nv_init.Set(nv, dual=True, definedon=mesh.Boundaries(".*"))
# Normalize averaged normal vector on initial configuration
cf_averaged_nv_init_norm = Normalize(averaged_nv_init)
# Project averaged unit normal vector being perpendicular to deformed edge-tangent vector
# to measure correct angle on deformed configuration
cf_proj_averaged_nv = Normalize(averaged_nv - (tv_def * averaged_nv) * tv_def)

Define the material and inverse material norms $\|\cdot\|_{\mathbb{M}}^2$ and $\|\cdot\|_{\mathbb{M}^{-1}}^2$ with Young modulus $\bar{E}$ and Poisson's ratio $\bar{\nu}$
\begin{align*}
\mathbb{M} \boldsymbol{E} = \frac{\bar E}{1-\bar \nu^2}\big((1-\bar \nu)\boldsymbol{E}+\bar \nu\,\mathrm{tr}(\boldsymbol{E})\boldsymbol{P}\big),\qquad\mathbb{M}^{-1} \boldsymbol{m} = \frac{1+\bar \nu}{\bar E}\big(\boldsymbol{m}-\frac{\bar \nu}{\bar\nu+1}\,\mathrm{tr}(\boldsymbol{m})\boldsymbol{P}\big).
\end{align*}

In [ ]:
# Material norm
def MaterialNorm(mat, E, nu):
    return E / (1 - nu**2) * ((1 - nu) * InnerProduct(mat, mat) + nu * Trace(mat) ** 2)


# Material stress
def MaterialStress(mat, E, nu):
    return E / (1 - nu**2) * ((1 - nu) * mat + nu * Trace(mat) * Ptau)


# Inverse of the material norm
def MaterialNormInv(mat, E, nu):
    return (1 + nu) / E * (InnerProduct(mat, mat) - nu / (nu + 1) * Trace(mat) ** 2)

Define shell energies, where the bending energy is incorporated as a saddle point problem. We set ``condense=True`` in the bilinear form to compute the Schur complement eliminating the bending moment tensor unknowns $\boldsymbol{m}$ from the global system. Thus, we obtain a minimization problem in $(u,\alpha)$.

In [ ]:
# Bilinear form for problem
# We define the Lagrangian of the TDNNS formulation. Therefore,
# we use Variation() such that Newton knows to build the first variation
# for the residual and the second variation for the stiffness matrix. The
# stiffness matrix  will be symmetric and we use static condensation via
# condense = True to eliminate the bending moment tensor m from the global system.
# We use .Compile() to simplify (linearize) the coefficient expression tree.

bfA = BilinearForm(fes, symmetric=True, condense=True)
# Membrane energy
# Interpolate the membrane strains Etautau into the Regge space
# to avoid membrane locking
bfA += Variation(
    0.5 * thickness * MaterialNorm(Interpolate(Etautau, Regge), E, nu) * ds
).Compile()
# Bending energy
# Element terms of bending energy
bfA += Variation(
    (
        -6 / thickness**3 * MaterialNormInv(m, E, nu)
        + InnerProduct(
            H_nv_def + (J - nv * director) * Grad(nv) - Grad(gamma), 1 / J * m
        )
    )
    * ds
).Compile()
# Boundary terms of bending energy including hybridization variable
bfA += Variation(
    (
        acos(cnv_def * cf_proj_averaged_nv)
        - acos(cnv * cf_averaged_nv_init_norm)
        + hyb * cnv
        + (PseudoInverse(Ftau, nv).trans * gamma) * cnv_def
    )
    * (m * cnv * cnv)
    * ds(element_boundary=True)
).Compile()
# Shear energy
bfA += Variation(0.5 * thickness * kappa * G * gamma * gamma * ds)

# Shear force. Parameter par for load-stepping below.
par = Parameter(0.0)
bfA += Variation(-par * force * u * ds("domain"))

In [ ]:
# Reset the solution to zero
gf_solution.vec[:] = 0
# Extract components of solution
gfu, gfm, gfgamma, _ = gf_solution.components

# Draw displacement (x10), bending moment tensor, shear stresses, and membrane stresses
scene = Draw(gfu, mesh, "displacement", deformation=10 * gfu)
scene2 = Draw(Norm(gfm), mesh, "bending_stress", deformation=gfu)
scene3 = Draw(thickness * kappa * G * gfgamma, mesh, "shear_stress", deformation=gfu)

Use Newton's method for solving and increase magnitude of right-hand side by load-steps.

The unit normal vector is averaged after each load-step.

In [ ]:
# Define the points of interest
point_P = (L / 2, L / 2, 0)
result_P = [(0, 0, 0)]

# Use num_steps uniform load-steps
num_steps = 15

# Thread parallel
with TaskManager():
    for steps in range(num_steps):
        par.Set((steps + 1) / num_steps)
        print("Loadstep =", steps + 1, ", F/Fmax =", (steps + 1) / num_steps * 100, "%")

        # Update averaged normal vector
        averaged_nv.Set(
            cf_nv_cur,
            dual=True,
            definedon=mesh.Boundaries(".*"),
        )

        # Use Newton solver with residual tolerance 1e-5 and maximal 100 iterations
        # Due to hybridization techniques we can use the sparsecholesky solver for
        # solving the Schur complement (done internally)
        solvers.Newton(
            bfA,
            gf_solution,
            inverse="sparsecholesky",
            printing=False,
            maxerr=1e-5,
            maxit=100,
        )
        # Redraw solutions
        scene.Redraw()
        scene2.Redraw()
        scene3.Redraw()

        result_P.append((gfu(mesh(*point_P, BND))))

Plot the solution.

In [ ]:
import matplotlib.pyplot as plt

_, _, P_z = zip(*result_P)
y_axis = [i / num_steps for i in range(len(P_z))]

plt.plot(P_z, y_axis, "-*", label="$P_z$")

plt.xlabel("displacement")
plt.ylabel("$P/P_{\\max}$")
plt.legend()
plt.show()